In [2]:
from keras import layers, models, losses, optimizers
import pandas as pd

In [8]:
quant_df = pd.read_csv('./quant_df.csv')

In [11]:
quant_df.head()

,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,...,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,TARGET
0,0,270000.0,675000.0,35082.0,675000.0,0.046220,-8879,-733,-8544.0,-1566,...,4.0,0.0,-583.0,0.0,0.0,0.0,0.0,0.0,3.0,0
1,0,81000.0,148500.0,10687.5,148500.0,0.028663,-23390,365243,-3184.0,-4289,...,0.0,0.0,-1116.0,0.0,0.0,0.0,0.0,0.0,4.0,0
2,0,225000.0,1314117.0,36265.5,1147500.0,0.031329,-17435,-7451,-11507.0,-733,...,0.0,0.0,-544.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,1,225000.0,1078200.0,38331.0,900000.0,0.010147,-10609,-1480,-2610.0,-3275,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1
4,2,135000.0,589500.0,25105.5,589500.0,0.015221,-13963,-792,-5411.0,-4645,...,0.0,0.0,-736.0,0.0,0.0,0.0,0.0,0.0,5.0,0


## Model

In [8]:
N_FEAT = 122
N_UNITS = [1024, 1024, 512]

In [16]:
input_ = layers.Input(shape=(N_FEAT, ), name="Input")

fc_1 = layers.Dense(units=N_UNITS[0], activation='relu', name='fc_1')(input_)
fc_2 = layers.Dense(units=N_UNITS[1], activation='relu', name='fc_2')(fc_1)
fc_3 = layers.Dense(units=N_UNITS[2], activation='relu', name='fc_3')(fc_2)
output = layers.Dense(units=1, activation='softmax', name='output')(fc_3)

model = models.Model(inputs=input_, outputs=output)
model.compile(loss=losses.binary_crossentropy, optimizer='adam')
